In [1]:
use_extracted_data = False

In [2]:
import glob
import os

import joblib
import numpy as np
import pandas as pd
from gplately import Raster

from lib.check_files import check_prepared_data
from lib.pu import create_grids


In [3]:
verbose = bool(int(os.environ.get("VERBOSE", 1)))
n_jobs = int(os.environ.get("N_JOBS", 8))

## Input and output files

In [4]:
# Input
if use_extracted_data:
    data_dir = "extracted_data"
else:
    data_dir = check_prepared_data("prepared_data")
data_filename = os.path.join(data_dir, "grid_data.csv")

outputs_dir = "outputs"
dist_parametric_filename = os.path.join(
    outputs_dir,
    "global",
    "erodep_dist_parametric.joblib",
)
dist_kde_filename = os.path.join(
    outputs_dir,
    "global",
    "erodep_dist_kde.joblib",
)

# Output
output_filename_points = os.path.join(
    outputs_dir,
    "global",
    "grid_data_preservation_likelihood.csv",
)
erodep_grid_dir = os.path.join(
    outputs_dir,
    "erosion_grids",
)
likelihood_parametric_dir = os.path.join(
    outputs_dir,
    "global",
    "preservation_likelihood_grids_parametric",
)
likelihood_kde_dir = os.path.join(
    outputs_dir,
    "global",
    "preservation_likelihood_grids_kde",
)
for i in (
    erodep_grid_dir,
    likelihood_parametric_dir,
    likelihood_kde_dir,
):
    os.makedirs(i, exist_ok=True)

In [5]:
cols = ["lon", "lat", "age (Ma)", "erosion (m)"]
data = pd.read_csv(data_filename, usecols=cols)

if verbose:
    print(
        "Loading parametric distribution from file: "
        + dist_parametric_filename
    )
dist_parametric = joblib.load(dist_parametric_filename)
data["likelihood_parametric"] = dist_parametric.pdf(data["erosion (m)"])

if verbose:
    print(
        "Loading KDE distribution from file: "
        + dist_kde_filename
    )
dist_kde = joblib.load(dist_kde_filename)
valid_inds_kde = data.index[~np.isnan(data["erosion (m)"])]
x_kde = np.reshape(data.loc[valid_inds_kde, "erosion (m)"], (-1, 1))
lh_kde = np.exp(dist_kde.score_samples(x_kde))
data["likelihood_kde"] = np.nan
for i, lh in zip(valid_inds_kde, lh_kde):
    data.at[i, "likelihood_kde"] = lh
del x_kde, lh_kde, valid_inds_kde

data.to_csv(output_filename_points, index=False)

## Create grids

### Erosion

In [6]:
create_grids(
    data,
    output_dir=erodep_grid_dir,
    threads=n_jobs,
    verbose=verbose,
    column="erosion (m)",
    filename_format="erosion_grid_{}Ma.nc",
    extent="global",
)


### Preservation likelihood

In [7]:
for erodep_filename in glob.glob(
    os.path.join(
        erodep_grid_dir,
        r"erosion_grid_*Ma.nc",
    )
):
    if verbose:
        print(
            f"Reading file: {os.path.basename(erodep_filename)}"
        )
    likelihood_basename = os.path.basename(
        erodep_filename
    ).replace("erosion_grid", "preservation_likelihood_grid")
    likelihood_parametric_filename = os.path.join(
        likelihood_parametric_dir,
        likelihood_basename,
    )
    likelihood_kde_filename = os.path.join(
        likelihood_kde_dir,
        likelihood_basename,
    )

    erodep_raster = Raster(erodep_filename)

    likelihood_parametric = Raster(dist_parametric.pdf(erodep_raster))
    if verbose:
        print(
            f" - Writing parametric file: {likelihood_basename}"
        )
    likelihood_parametric.save_to_netcdf4(likelihood_parametric_filename)
    del likelihood_parametric

    likelihood_kde = np.full_like(erodep_raster.data, np.nan)
    mask_kde = np.isnan(np.array(erodep_raster.data))
    if mask_kde.mean() < 1.0:
        kde_input = np.reshape(
            np.array(erodep_raster.data[~mask_kde]),
            (-1, 1),
        )
        likelihood_kde[~mask_kde] = np.ravel(
            np.exp(
                dist_kde.score_samples(kde_input)
            )
        )
    likelihood_kde = Raster(likelihood_kde)
    if verbose:
        print(
            f" - Writing KDE file: {likelihood_basename}"
        )
    likelihood_kde.save_to_netcdf4(likelihood_kde_filename)
    del likelihood_kde